In [1]:
%load_ext autoreload

In [2]:
import sys
import os
import time
import numpy as np
from functools import partial
import itertools
import mcubes
import visualizations_utils as viz_utils
import iris_utils #TODO remove
from iris_plant_visualizer import IrisPlantVisualizer

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.all import InverseKinematics, RevoluteJoint
import pydrake.symbolic as sym
from pydrake.all import MathematicalProgram, RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

import meshcat

# Build plant

In [4]:

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

simple_collision = True
use_gripper = False
if simple_collision and use_gripper:
    directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_welded_gripper.yaml")
elif not simple_collision and use_gripper:
    directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_welded_gripper.yaml") 
elif simple_collision and not use_gripper:
    directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_simple_collision_no_gripper.yaml")
elif not simple_collision and not use_gripper:
    directives_file = FindResourceOrThrow("drake/sos_iris_certifier/planar_iiwa_dense_collision_no_gripper.yaml") 
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()
# visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, 
#                                       delete_prefix_on_load=False)

# diagram = builder.Build()
# visualizer.load()


# Setup meshcat visualization

In [5]:
do_viz = True
visualizer = IrisPlantVisualizer(plant, builder, scene_graph)
diagram = visualizer.diagram

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6004...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7004/static/
Connected to meshcat-server.


# Run SNOPT IRIS in rational space

In [6]:
iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 5
iris_options.configuration_space_margin = 1e-4
iris_options.termination_threshold = -1
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False
iris_options.certify_region_with_sos_during_generation = False
iris_options.certify_region_with_sos_after_generation = False
# uncomment to test if non-default is working
# currently throwing std::bad_alloc if we try to set q_star
iris_options.q_star = 0.5 * np.zeros(3)
rational_forward_kinematics = RationalForwardKinematics(plant)


#seed points specified in q space
seed_points_q = np.array([
                        [0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]      # out of shelf 1
                        ]) 
seed_points_t = np.array([rational_forward_kinematics.ComputeTValue(s, iris_options.q_star) for s in seed_points_q])

t_lower = rational_forward_kinematics.ComputeTValue(plant.GetPositionLowerLimits(),iris_options.q_star)
t_upper = rational_forward_kinematics.ComputeTValue(plant.GetPositionUpperLimits(),iris_options.q_star)
P_joint_limits = HPolyhedron.MakeBox(t_lower, t_upper)

In [7]:
regions = []
context = diagram.CreateDefaultContext()
for i, s in enumerate(seed_points_q):
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), s)
    if i > 0:
        starting_hpolyhedron = regions[i-1]
        r = IrisInRationalConfigurationSpace (plant, plant.GetMyContextFromRoot(context),
                                              iris_options, starting_hpolyhedron)
    else:
        r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    regions.append(r)
    print(f'Completed region: {i+1}/{len(seed_points_q)}')

Completed region: 1/6
Completed region: 2/6
Completed region: 3/6
Completed region: 4/6
Completed region: 5/6
Completed region: 6/6


In [8]:
# plot regions and collision constraint
if do_viz:
    visualizer.plot_regions(regions)
    visualizer.plot_seedpoints(seed_points_t)
    visualizer.visualize_collision_constraint(N = 50)

# Begin Certification

In [9]:
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.solvers import mathematicalprogram as mp
cspace_free_region = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)

In [12]:
editted_regions = []
filtered_collision_pairs = set()

binary_search_options = rational_forward_kinematics.BinarySearchOption()
binary_search_options.epsilon_max = 10
binary_search_options.epsilon_min = -.2
binary_search_options.max_iters = 5
binary_search_options.search_d = True

bilinear_alternation_option = rational_forward_kinematics.BilinearAlternationOption()
bilinear_alternation_option.max_iters = 20
bilinear_alternation_option.lagrangian_backoff_scale = 1e-3
bilinear_alternation_option.polytope_backoff_scale = 1e-5

solver_options = mp.SolverOptions()
for r in regions: 
    try:
        d_feasible = \
            cspace_free_region.CspacePolytopeBinarySearch(
                iris_options.q_star, filtered_collision_pairs, r.A(), r.b(),
                binary_search_options, solver_options)
        C_final, d_final, P_final, q_final = \
            cspace_free_region.CspacePolytopeBilinearAlternation(
                iris_options.q_star, filtered_collision_pairs, r.A(), d_feasible,
                bilinear_alternation_option, solver_options)
        C_final = np.vstack([C_final, P_joint_limits.A()])
        d_final = np.concatenate([d_final, P_joint_limits.b()])
        editted_regions.append(HPolyhedron(C_final, d_final))
    except Exception as e:
        print(e)
print(len(editted_regions))

[2022-01-21 11:09:04.678] [console] [info] Solver time 0.043287038803100586
[2022-01-21 11:09:04.899] [console] [info] Solver time 0.050316810607910156
[2022-01-21 11:09:05.085] [console] [info] Solver time 0.04792189598083496
[2022-01-21 11:09:05.086] [console] [info] epsilon=4.9 is infeasible
[2022-01-21 11:09:05.276] [console] [info] Solver time 0.06190991401672363
[2022-01-21 11:09:05.277] [console] [info] epsilon=2.35 is infeasible
[2022-01-21 11:09:05.472] [console] [info] Solver time 0.059155941009521484
[2022-01-21 11:09:05.474] [console] [info] epsilon=1.075 is infeasible
[2022-01-21 11:09:05.662] [console] [info] Solver time 0.06271600723266602
[2022-01-21 11:09:05.664] [console] [info] epsilon=0.4375 is infeasible
[2022-01-21 11:09:05.862] [console] [info] Solver time 0.0653538703918457
[2022-01-21 11:09:05.864] [console] [info] epsilon=0.11875 is infeasible
[2022-01-21 11:09:06.096] [console] [info] Iter: 0, max(log(det(P)))=-0.03320937797731179, solver_time 0.1383960247039

[2022-01-21 11:09:13.718] [console] [info] backoff with log(det(P)) 0.8053514059309257, solver time 0.10217809677124023
[2022-01-21 11:09:13.719] [console] [info] cost improvement 0.006911176612026226
[2022-01-21 11:09:13.838] [console] [info] Iter: 13, polytope step cost -0.4372124874321673, solver time 0.05628180503845215
[2022-01-21 11:09:13.915] [console] [info] back off, cost: -0.43720812044178214, solver time 0.06710004806518555
[2022-01-21 11:09:14.134] [console] [info] Iter: 14, max(log(det(P)))=0.8128949946486901, solver_time 0.12323212623596191
[2022-01-21 11:09:14.255] [console] [info] backoff with log(det(P)) 0.8121543762272465, solver time 0.10542106628417969
[2022-01-21 11:09:14.256] [console] [info] cost improvement 0.006809511397193324
[2022-01-21 11:09:14.366] [console] [info] Iter: 14, polytope step cost -0.448585746166204, solver time 0.04746413230895996
[2022-01-21 11:09:14.496] [console] [info] back off, cost: -0.44858127261623526, solver time 0.11896300315856934
[

[2022-01-21 11:09:22.062] [console] [info] cost improvement 0.003459384718922598
[2022-01-21 11:09:22.162] [console] [info] Iter: 6, polytope step cost -0.3005466823823261, solver time 0.04428696632385254
[2022-01-21 11:09:22.296] [console] [info] back off, cost: -0.30054369288229876, solver time 0.12403988838195801
[2022-01-21 11:09:22.508] [console] [info] Iter: 7, max(log(det(P)))=0.9974867219639894, solver_time 0.12737202644348145
[2022-01-21 11:09:22.633] [console] [info] backoff with log(det(P)) 0.9965826054481102, solver time 0.10996103286743164
[2022-01-21 11:09:22.633] [console] [info] cost improvement 0.0030458235145455204
[2022-01-21 11:09:22.730] [console] [info] Iter: 7, polytope step cost -0.31236749763598665, solver time 0.041552066802978516
[2022-01-21 11:09:22.818] [console] [info] back off, cost: -0.3123643775583738, solver time 0.07788586616516113
[2022-01-21 11:09:23.025] [console] [info] Iter: 8, max(log(det(P)))=1.0003502819953873, solver_time 0.12260603904724121


[2022-01-21 11:09:30.240] [console] [info] epsilon=0.4375 is infeasible
[2022-01-21 11:09:30.422] [console] [info] Solver time 0.06615400314331055
[2022-01-21 11:09:30.423] [console] [info] epsilon=0.11875 is infeasible
[2022-01-21 11:09:30.631] [console] [info] Iter: 0, max(log(det(P)))=0.6136718689435359, solver_time 0.1224679946899414
[2022-01-21 11:09:30.752] [console] [info] backoff with log(det(P)) 0.6131108317952594, solver time 0.10710310935974121
[2022-01-21 11:09:30.753] [console] [info] cost improvement inf
[2022-01-21 11:09:30.854] [console] [info] Iter: 0, polytope step cost -0.14655077899523275, solver time 0.04462790489196777
[2022-01-21 11:09:30.929] [console] [info] back off, cost: -0.14654931365612467, solver time 0.06448221206665039
[2022-01-21 11:09:31.148] [console] [info] Iter: 1, max(log(det(P)))=0.8458827261360996, solver_time 0.1351320743560791
[2022-01-21 11:09:31.271] [console] [info] backoff with log(det(P)) 0.845117201236697, solver time 0.10903501510620117

[2022-01-21 11:09:38.149] [console] [info] cost improvement 0.0020553438567043614
[2022-01-21 11:09:38.248] [console] [info] Iter: 14, polytope step cost -0.2767016955691024, solver time 0.04332399368286133
[2022-01-21 11:09:38.319] [console] [info] back off, cost: -0.27669898882779176, solver time 0.06054186820983887
[2022-01-21 11:09:38.557] [console] [info] Iter: 15, max(log(det(P)))=1.0124910653066617, solver_time 0.1530289649963379
[2022-01-21 11:09:38.679] [console] [info] backoff with log(det(P)) 1.0115560518290732, solver time 0.10688090324401855
[2022-01-21 11:09:38.679] [console] [info] cost improvement 0.0019470545327655575
[2022-01-21 11:09:38.786] [console] [info] Iter: 15, polytope step cost -0.2760942641529159, solver time 0.050360918045043945
[2022-01-21 11:09:38.911] [console] [info] back off, cost: -0.2760915118172158, solver time 0.11408495903015137
[2022-01-21 11:09:39.128] [console] [info] Iter: 16, max(log(det(P)))=1.0143348885067889, solver_time 0.131407976150512

[2022-01-21 11:09:46.955] [console] [info] Iter: 7, polytope step cost -0.22915032438721292, solver time 0.05077695846557617
[2022-01-21 11:09:47.058] [console] [info] back off, cost: -0.22914803846870646, solver time 0.09127211570739746
[2022-01-21 11:09:47.303] [console] [info] Iter: 8, max(log(det(P)))=0.36613325186883827, solver_time 0.14713621139526367
[2022-01-21 11:09:47.445] [console] [info] backoff with log(det(P)) 0.36580373977493363, solver time 0.12469696998596191
[2022-01-21 11:09:47.445] [console] [info] cost improvement 0.20458419849429146
[2022-01-21 11:09:47.596] [console] [info] Iter: 8, polytope step cost -0.266300381190767, solver time 0.0869297981262207
[2022-01-21 11:09:47.666] [console] [info] back off, cost: -0.26629772327583234, solver time 0.05870699882507324
[2022-01-21 11:09:47.899] [console] [info] Iter: 9, max(log(det(P)))=0.5926508859133586, solver_time 0.14194607734680176
[2022-01-21 11:09:48.042] [console] [info] backoff with log(det(P)) 0.5921140066184

[2022-01-21 11:09:56.009] [console] [info] back off, cost: -0.17453809087327046, solver time 0.06182408332824707
[2022-01-21 11:09:56.230] [console] [info] Iter: 6, max(log(det(P)))=0.9888011493557607, solver_time 0.1290760040283203
[2022-01-21 11:09:56.358] [console] [info] backoff with log(det(P)) 0.987908484734328, solver time 0.11225509643554688
[2022-01-21 11:09:56.358] [console] [info] cost improvement 0.004792214741911605
[2022-01-21 11:09:56.507] [console] [info] Iter: 6, polytope step cost -0.1650474612524257, solver time 0.08553910255432129
[2022-01-21 11:09:56.574] [console] [info] back off, cost: -0.16504581211785177, solver time 0.056210994720458984
[2022-01-21 11:09:56.807] [console] [info] Iter: 7, max(log(det(P)))=0.9934827010355977, solver_time 0.14056992530822754
[2022-01-21 11:09:56.937] [console] [info] backoff with log(det(P)) 0.9925839414183031, solver time 0.11514091491699219
[2022-01-21 11:09:56.938] [console] [info] cost improvement 0.0046618284300571355
[2022-

[2022-01-21 11:10:05.035] [console] [info] Solver time 0.04759788513183594
[2022-01-21 11:10:05.037] [console] [info] epsilon=2.35 is infeasible
[2022-01-21 11:10:05.214] [console] [info] Solver time 0.05361294746398926
[2022-01-21 11:10:05.216] [console] [info] epsilon=1.075 is infeasible
[2022-01-21 11:10:05.400] [console] [info] Solver time 0.06312704086303711
[2022-01-21 11:10:05.401] [console] [info] epsilon=0.4375 is infeasible
[2022-01-21 11:10:05.601] [console] [info] Solver time 0.07796597480773926
[2022-01-21 11:10:05.603] [console] [info] epsilon=0.11875 is infeasible
[2022-01-21 11:10:05.796] [console] [info] Iter: 0, max(log(det(P)))=0.5796918670084601, solver_time 0.10041189193725586
[2022-01-21 11:10:05.914] [console] [info] backoff with log(det(P)) 0.579184219036567, solver time 0.10242986679077148
[2022-01-21 11:10:05.914] [console] [info] cost improvement inf
[2022-01-21 11:10:06.019] [console] [info] Iter: 0, polytope step cost -0.1558232818430138, solver time 0.0425

[2022-01-21 11:10:13.418] [console] [info] back off, cost: -0.15190431366528417, solver time 0.0704488754272461
[2022-01-21 11:10:13.648] [console] [info] Iter: 14, max(log(det(P)))=1.0077768431547955, solver_time 0.1367480754852295
[2022-01-21 11:10:13.778] [console] [info] backoff with log(det(P)) 1.0068728532845168, solver time 0.11459708213806152
[2022-01-21 11:10:13.779] [console] [info] cost improvement 0.0024311704336530404
[2022-01-21 11:10:13.896] [console] [info] Iter: 14, polytope step cost -0.15463331245927978, solver time 0.05424904823303223
[2022-01-21 11:10:13.988] [console] [info] back off, cost: -0.1546317775608937, solver time 0.08159589767456055
[2022-01-21 11:10:14.212] [console] [info] Iter: 15, max(log(det(P)))=1.0101266843009342, solver_time 0.12968111038208008
[2022-01-21 11:10:14.340] [console] [info] backoff with log(det(P)) 1.0092192780177167, solver time 0.11193203926086426
[2022-01-21 11:10:14.340] [console] [info] cost improvement 0.0023504493559749484
[20

6


In [13]:
if do_viz:
    visualizer.plot_regions(editted_regions, region_suffix = 'editted')